# HOW TO ADD NEW SPECIES TO THE PHOTOLYSIS MODEL

Created by Carlos Gómez de Olea B., September 2025 as part of the requirements for his Master's Thesis at TU Munich's Professorship of Lunar and Planetary Exploration.

## Main Concept

The Exospheric Photolysis model is configured to support only a limited number of atomic and molecular species by default. These include:

- Water ($H_2O$)  
- Hydroxyl ($OH$)  
- Molecular Hydrogen ($H_2$)  
- Atomic Hydrogen ($H$)  
- Ionized (negative) Hydrogen ($H^{-}$)  
- Hydroperoxyl ($HO_2$)  
- Hydrogen Peroxide ($H_2O_2$)  
- Helium ($He$)  
- Neon ($Ne$)  

So far, the model has been applied primarily for investigations involving the Moon. This explains the focus on hydrogen- and oxygen-bearing species (for studying the lunar water cycle) and noble gases such as Neon and Helium, which are the main constituents of the Moon's exosphere.  

Technically, any species of interest could be added to the exospheric model for other lunar investigations or for studying potential exospheres of other celestial bodies. In this notebook, two methods are suggested for adding new species.

## Example: Helium

In [ ]:
import numpy as np
import pandas as pd

The following function provides normalized values of the solar flux in a certain wavelength range. This is equivalent to the weights or probabilities of a photon incoming from the sun having a certain energy/wavelength per unit time for a specific species (the flux is cross-sectiond dependent!)

In [2]:
def normalize_flux_distribution(wavelengths, quiet_fluxes, min_wavelength, max_wavelength):
    """
    Function to normalize solar flux in a certain wavelength range.

    Parameters:
    wavelengths (list or np.array): List or array of wavelength values.
    quiet_fluxes (list or np.array): List or array of flux values corresponding to the wavelengths.
    min_wavelength (float): Minimum wavelength of the range.
    max_wavelength (float): Maximum wavelength of the range.

    Returns:
    filtered_wavelengths (np.array): Array of wavelengths within the specified range.
    normalized_fluxes (np.array): Normalized flux values within the specified range.
    """
    
    # Convert to numpy arrays if they are lists
    wavelengths = np.array(wavelengths)
    quiet_fluxes = np.array(quiet_fluxes)

    # Step 1: Filter data to get wavelengths within the specified range
    filtered_indices = (wavelengths >= min_wavelength) & (wavelengths <= max_wavelength)
    filtered_wavelengths = wavelengths[filtered_indices]
    filtered_fluxes = quiet_fluxes[filtered_indices]

    # Step 2: Exclude Lyman-alpha if the condition holds
    if min_wavelength != 0.0 and max_wavelength != 95000.0:
        if min_wavelength < 1208.0 and max_wavelength > 1220.0:
            exclude_indices = filtered_wavelengths != 1212  # Exclude 1220 nm
            filtered_wavelengths = filtered_wavelengths[exclude_indices]
            filtered_fluxes = filtered_fluxes[exclude_indices]

    # Step 3: Normalize the fluxes
    total_flux = np.sum(filtered_fluxes)
    normalized_fluxes = filtered_fluxes / total_flux if total_flux != 0 else filtered_fluxes
    
    return filtered_wavelengths, normalized_fluxes

#### Solar flux expressed in $cm^{-2} s^{-1}$ per wavelength bin from Huebner (2015)

In [3]:
# Wavelength values
lambda_phi = [
    1, 2, 4, 6, 8, 10, 40, 50, 60, 70, 80, 90, 100, 110, 120, 128, 153, 176, 205, 231, 270, 280, 300, 320, 340, 360, 370, 400, 430, 460, 480, 500, 
    520, 540, 560, 580, 600, 620, 630, 680, 710, 740, 770, 800, 830, 860, 890, 911, 920, 930, 940, 950, 960, 970, 980, 995, 1007, 1017, 1027, 1035, 1045, 1057, 1070, 1080, 
    1090, 1100, 1110, 1120, 1130, 1140, 1150, 1157, 1163, 1170, 1176, 1183, 1190, 1198, 1205, 1212, 1220, 1227, 1235, 1242, 1250, 1258, 1266, 1274, 1282, 1290, 1299, 1307, 1316, 1324, 1333, 1342,
    1351, 1360, 1370, 1379, 1389, 1408, 1428, 1449, 1470, 1492, 1515, 1538, 1562, 1587, 1613, 1639, 1667, 1695, 1724, 1739, 1754, 1770, 1786, 1802, 1818, 1835, 1852, 1869, 1887, 1905, 1923, 1942, 
    1961, 1980, 2000, 2020, 2041, 2062, 2083, 2105, 2128, 2150, 2174, 2198, 2222, 2247, 2273, 2299, 2326, 2353, 2381, 2410, 2439, 2469, 2500, 2532, 2564, 2597, 2632, 2667, 2703, 2740, 2778, 2817,
    2857, 2899, 2941, 2985, 3030, 3077, 3125, 3175, 3225, 3275, 3325, 3375, 3425, 3475, 3525, 3575, 3625, 3675, 3725, 3775, 3825, 3875, 3925, 3975, 4025, 4075, 4125, 4175, 4225, 4275, 4325, 4375, 4425, 
    4475, 4525, 4575, 4625, 4675, 4725, 4775, 4825, 4875, 4925, 4975, 5025, 5075, 5125, 5175, 5225, 5275, 5325, 5375, 5425, 5475, 5525, 5575, 5625, 5675, 5725, 5775, 5825, 5875, 5925, 5975, 6025, 6075, 
    6125, 6175, 6225, 6275, 6325, 6375, 6425, 6475, 6550, 6650, 6750, 6850, 6950, 7050, 7150, 7250, 7350, 7450, 7550, 7650, 7750, 7850, 7950, 8050, 8150, 8250, 8350, 8450, 8550, 8650, 8750, 8850, 8950, 
    9050, 9150, 9250, 9350, 9450, 9550, 9650, 9750, 9850, 9950, 10150, 10750, 11250, 11750, 12250, 12750, 13250, 13750, 14250, 14750, 15250, 15750, 16250, 16750, 17250, 17750, 18250, 18750, 19250, 19750, 20375, 21500, 22500, 
    23500, 24500, 25500, 26500, 27500, 28500, 29500, 30500, 31500, 32500, 33500, 34500, 35500, 36500, 37500, 38500, 39500, 40500, 41500, 42500, 43500, 44500, 45500, 46500, 47500, 48500, 49500, 52750, 65000, 75000, 85000, 95000
]

# Flux in cm^-2 s^-1 bin^-1 (scientific notation) from Huebner (2015)
phi = [
    0.0, 3.0e1, 2.5e3, 2.8e4, 1.8e5, 4.0e6, 4.7e7, 8.3e7, 1.03e8, 9.4e7, 1.2e8, 9.9e7, 5.6e7, 2.5e7, 1.2e7, 3.07e8, 9.0e8, 3.7e9, 1.4e9, 2.65e9, 4.5e8, 1.54e9, 5.84e9, 1.16e9, 4.5e8, 1.31e9, 1.77e8, 2.76e8, 1.77e8, 2.12e8, 4.32e8, 6.32e8, 
    2.42e8, 3.62e8, 5.17e7, 1.02e9, 5.52e8, 1.2e9, 1.15e8, 3.39e8, 1.59e8, 5.19e8, 1.28e9, 8.07e8, 1.05e9, 3.73e9, 3.34e9, 9.39e7, 2.34e8, 5.84e8, 5.94e8, 1.04e8, 1.04e8, 5.3e9, 7.56e8, 1.25e8, 1.04e8, 3.6e9, 2.46e9, 1.83e9, 1.57e8, 1.7e8, 1.31e8, 7.21e8, 
    1.31e8, 1.31e8, 1.31e8, 6.21e8, 1.31e8, 1.31e8, 9.15e7, 7.84e7, 1.03e8, 2.66e8, 1.12e8, 1.24e8, 1.82e8, 1.9e8, 7.4e8, 3.02e11, 3.67e9, 1.36e9, 1.61e9, 1.32e9, 1.41e9, 3.11e9, 1.06e9, 1.37e9, 1.02e9, 1.14e9, 7.29e9, 2.2e9, 1.59e9, 2.21e9, 1.24e9, 1.99e9, 
    3.09e9, 2.57e9, 2.74e9, 3.1e9, 7.6e9, 1.01e10, 1.3e10, 1.82e10, 1.33e10, 1.66e10, 2.9e10, 3.6e10, 4.75e10, 6.4e10, 5.49e10, 1.19e11, 1.76e11, 2.32e11, 1.44e11, 1.83e11, 1.34e11, 2.62e11, 2.88e11, 3.14e11, 3.81e11, 4.43e11, 4.95e11, 5.94e11, 6.59e11, 7.26e11, 9.85e11, 1.27e12, 
    1.01e12, 1.2e12, 1.44e12, 1.8e12, 2.08e12, 2.45e12, 5.09e12, 7.12e12, 9.23e12, 8.42e12, 1.2e13, 1.22e13, 1.77e13, 1.6e13, 1.96e13, 2.4e13, 2.25e13, 2.21e13, 2.32e13, 2.5e13, 2.73e13, 2.88e13, 3.02e13, 3.97e13, 7.13e13, 4.37e13, 1.12e14, 1.25e14, 1.16e14, 1.19e14, 1.38e14, 1.70e14,
    2.46e14, 3.90e14, 3.99e14, 3.86e14, 5.08e14, 5.92e14, 6.05e14, 6.94e14, 8.12e14, 9.71e14, 8.97e14, 9.44e14, 1.01e15, 1.03e15, 1.03e15, 1.04e15, 1.18e15, 1.23e15, 1.24e15, 1.17e15, 1.11e15, 1.09e15, 1.19e15, 1.54e15, 1.90e15, 1.99e15, 1.99e15, 2.02e15, 2.01e15, 1.94e15, 1.98e15, 2.25e15, 2.39e15, 
    2.48e15, 2.49e15, 2.48e15, 2.50e15, 2.55e15, 2.61e15, 2.59e15, 2.46e15, 2.44e15, 2.53e15, 2.48e15, 2.49e15, 2.50e15, 2.43e15, 2.43e15, 2.52e15, 2.58e15, 2.64e15, 2.67e15, 2.70e15, 2.68e15, 2.66e15, 2.66e15, 2.67e15, 2.67e15, 2.69e15, 2.71e15, 2.71e15, 2.71e15, 2.72e15, 2.72e15, 2.71e15, 2.70e15,
    2.70e15, 2.70e15, 2.69e15, 2.68e15, 2.67e15, 2.66e15, 2.65e15, 3.95e15, 5.22e15, 5.18e15, 5.14e15, 5.09e15, 5.04e15, 4.99e15, 4.94e15, 4.90e15, 4.83e15, 4.80e15, 4.67e15, 4.59e15, 4.68e15, 4.54e15, 4.60e15, 4.53e15, 4.41e15, 4.34e15, 4.31e15, 4.03e15, 4.33e15, 4.25e15, 4.27e15, 4.23e15, 4.13e15, 
    4.01e15, 3.89e15, 3.88e15, 3.78e15, 3.76e15, 3.72e15, 3.72e15, 3.79e15, 3.79e15, 7.49e15, 2.11e16, 1.68e16, 1.60e16, 1.50e16, 1.48e16, 1.43e16, 1.32e16, 1.25e16, 1.16e16, 1.12e16, 1.07e16, 9.95e15, 9.72e15, 9.20e15, 8.24e15, 7.70e15, 6.94e15, 6.51e15, 6.19e15, 7.46e15, 1.11e16, 8.28e15, 7.33e15, 
    6.83e15, 6.07e15, 5.50e15, 4.96e15, 4.51e15, 4.09e15, 3.74e15, 3.40e15, 3.18e15, 2.87e15, 2.70e15, 2.47e15, 2.31e15, 2.14e15, 2.01e15, 1.87e15, 1.71e15, 1.60e15, 1.48e15, 1.41e15, 1.33e15, 1.25e15, 1.16e15, 1.06e15, 9.67e14, 9.25e14, 2.84e15, 6.74e15, 3.35e15, 2.22e15, 1.59e15, 4.53e15
]

#### Solar flux ratio between active and quiet sun conditions from Huebner (1992)

Extracted from Figure 3 with PlotDigitizer: https://plotdigitizer.com/

In [4]:
# Wavelength vector
lambda_active = [0.00, 28.72, 53.29, 59.16, 59.16, 71.54, 71.54, 74.44, 88.08, 89.99, 
115.77, 115.77, 125.25, 125.25, 144.88, 144.88, 148.61, 148.61, 165.67, 176.91, 
223.91, 231.74, 272.68, 283.07, 298.20, 298.20, 300.99, 342.41, 344.49, 365.70, 
365.70, 366.69, 424.97, 426.87, 479.82, 480.91, 501.09, 534.21, 541.24, 580.65, 
585.59, 606.49, 609.04, 630.15, 633.66, 678.40, 683.51, 712.99, 714.07, 739.78, 
741.06, 769.47, 785.37, 796.68, 836.69, 922.60, 922.60, 961.07, 963.04, 970.42, 
1022.55, 1022.55, 1028.79, 1073.51, 1074.25, 1126.39, 1131.24, 1175.13, 1180.71, 
1184.81, 1224.35, 1271.98, 1278.65, 1300.04, 1318.69, 1326.50, 1357.78, 1364.96, 
1368.52, 1384.26, 1384.26, 1396.93, 1480.30, 1489.65, 1529.55, 1557.35, 1581.17, 
1610.21, 1615.05, 1684.84, 1687.57, 1786.33, 1918.57, 1938.65, 2034.93, 2049.99, 
2080.65, 2116.71, 2138.45, 2462.53, 2500.0, 95000.0]


# Ratio of flux active/quiet sun from Huebner (1992). Taken from Graph
ratio = [6.37, 6.37, 6.37, 2.50, 2.50, 2.32, 2.32, 2.48, 2.12, 2.31, 
2.35, 2.35, 2.58, 2.58, 3.60, 3.60, 4.51, 4.51, 4.51, 5.38, 
2.11, 5.37, 2.54, 2.10, 2.54, 2.54, 2.95, 2.14, 2.95, 2.83, 
2.83, 2.14, 2.41, 2.40, 2.40, 2.30, 2.27, 2.21, 2.01, 1.88, 
2.00, 1.79, 1.88, 1.94, 1.79, 1.94, 2.05, 2.18, 2.04, 1.93, 
2.18, 2.48, 1.91, 2.49, 2.41, 2.41, 1.97, 1.96, 2.14, 2.35, 2.36, 
1.98, 1.72, 1.72, 1.84, 1.84, 1.92, 1.84, 1.93, 2.34, 2.15, 
2.11, 1.87, 1.86, 1.88, 2.07, 1.92, 2.06, 1.99, 1.89, 1.89, 
1.65, 1.69, 1.55, 1.49, 1.48, 1.43, 1.42, 1.37, 1.26, 1.21, 
1.16, 1.15, 1.13, 1.08, 1.09, 1.05, 1.05, 1.03, 1.03, 1.01, 1.0]

#### Interpolated ratios for the wavelength bins given in Huebner (1992)

In [5]:
ratio_interp = np.interp(lambda_phi, lambda_active, ratio)

### Method 1: Cross sectional database is available per wavelength value

Example: Helium cross-sectional data from  Marr, West (1976)

In [6]:
He_CS_data = [(505.0, 1e-20),
    (504.0, 7.56), (500.0, 7.46), (495.0, 7.33), (490.0, 7.19), (485.0, 7.06), (480.0, 6.94),
    (475.0, 6.81), (470.0, 6.68), (465.0, 6.55), (460.0, 6.43), (455.0, 6.30),
    (450.0, 6.18), (445.0, 6.05), (440.0, 5.93), (435.0, 5.81), (430.0, 5.69),
    (425.0, 5.57), (420.0, 5.45), (415.0, 5.33), (410.0, 5.21), (405.0, 5.10),
    (400.0, 4.98), (395.0, 4.87), (390.0, 4.76), (385.0, 4.64), (380.0, 4.53),
    (375.0, 4.42), (370.0, 4.31), (365.0, 4.20), (360.0, 4.09), (355.0, 4.00),
    (350.0, 3.88), (345.0, 3.78), (340.0, 3.68), (335.0, 3.57), (330.0, 3.47),
    (325.0, 3.37), (320.0, 3.27), (315.0, 3.18), (310.0, 3.08), (305.0, 2.98),
    
    (300.0, 2.89), (295.0, 2.80), (290.0, 2.70), (285.0, 2.61), (280.0, 2.52),
    (275.0, 2.44), (270.0, 2.35), (265.0, 2.26), (260.0, 2.18), (255.0, 2.10),
    (250.0, 2.02), (245.0, 1.94), (240.0, 1.86), (235.0, 1.78), (230.0, 1.70),
    (225.0, 1.63), (220.0, 1.55), (215.0, 1.48), (210.0, 1.41), (205.0, 1.34),
    (200.0, 1.28), (195.0, 1.21), (190.0, 1.14), (185.0, 1.08), (180.0, 1.02),
    (175.0, 0.961), (170.0, 0.903), (165.0, 0.847), (160.0, 0.792), (155.0, 0.738),
    (150.0, 0.687), (145.0, 0.637), (140.0, 0.588), (135.0, 0.542), (130.0, 0.497),
    (125.0, 0.454), (120.0, 0.412), (115.0, 0.373), (110.0, 0.335), (105.0, 0.299), (100.0, 0.265), 
    
    (95.0, 0.233), (90.0, 0.202), ( 85.0, 0.174), ( 80.0, 0.147), ( 75.0, 0.123), 
    (70.0, 0.100), (65.0, 0.0795), ( 60.0, 0.0609),( 55.0, 0.0443), ( 50.0, 0.0315), 
    (48.0, 0.0282), (46.0, 0.0250),  (44.0, 0.0220), (42.0, 0.0193), (40.0, 0.0168), 
    (38.0, 0.0145), (36.0, 0.0124), (34.0, 0.0105), (32.0, 0.00885), (30.0, 0.00736), 
    (28.0, 0.00604), (26.0, 0.00489), (24.0, 0.00389), (22.0, 0.00303), (20.0, 0.00231), 
    (19.0, 0.00199), (18.0, 0.00171), (17.0, 0.00145), (16.0, 0.00122),(15.0, 0.00101), 
    (14.0, 0.000832), (13.0, 0.000673), (12.0, 0.000535), (11.0, 0.000417), (10.0, 0.000318), 
    (9.5, 0.000274), (9.0, 0.000235), (8.5, 0.000200), (8.0, 0.000168), (7.5, 0.000139), (7.0, 0.000115), 
    
    (6.5, 0.000093), (6.0, 0.000074), (5.5, 0.000057), (5.0, 0.000044), (4.5, 0.000031), 
    (4.0, 0.000023), (3.5, 0.000016), (3.0, 0.000010), (2.5, 0.000006), (2.0, 0.000003), (1.5, 0.000001), (0.0, 1e-20)
]

# Separate into two Python lists: wavelengths and cross sections
wvl = np.array([row[0] for row in He_CS_data])[::-1]
cs = np.array([row[1] for row in He_CS_data])[::-1]

#### Interpolate cross sectional data to have it in the same reference wvl as the solar flux

In [7]:
cs_interp = np.interp(lambda_phi, wvl, cs)

#### Quiet Sun

Calculate flux in photons/second and print photodestruction rate

In [8]:
He_flux_quiet = cs_interp * 1e-18 * phi

print("Photodestruction rate:", np.sum(He_flux_quiet))

Photodestruction rate: 5.574421370437615e-08


Print flux and copy it to _solar_spectrum.jl_ accordingly

In [9]:
He_flux_quiet

array([0.000000e+00, 9.000000e-23, 5.750000e-20, 2.072000e-18,
       3.024000e-17, 1.272000e-15, 7.896000e-13, 2.614500e-12,
       6.272700e-12, 9.400000e-12, 1.764000e-11, 1.999800e-11,
       1.484000e-11, 8.375000e-12, 4.944000e-12, 1.472986e-10,
       6.458400e-10, 3.599360e-09, 1.876000e-09, 4.547400e-09,
       1.057500e-09, 3.880800e-09, 1.687760e-08, 3.793200e-09,
       1.656000e-09, 5.357900e-09, 7.628700e-10, 1.374480e-09,
       1.007130e-09, 1.363160e-09, 2.998080e-09, 4.714720e-09,
       2.420000e-30, 3.620000e-30, 5.170000e-31, 1.020000e-29,
       5.520000e-30, 1.200000e-29, 1.150000e-30, 3.390000e-30,
       1.590000e-30, 5.190000e-30, 1.280000e-29, 8.070000e-30,
       1.050000e-29, 3.730000e-29, 3.340000e-29, 9.390000e-31,
       2.340000e-30, 5.840000e-30, 5.940000e-30, 1.040000e-30,
       1.040000e-30, 5.300000e-29, 7.560000e-30, 1.250000e-30,
       1.040000e-30, 3.600000e-29, 2.460000e-29, 1.830000e-29,
       1.570000e-30, 1.700000e-30, 1.310000e-30, 7.2100

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify _solar_spectrum.jl_ accordingly

In [10]:
normalize_flux_distribution(lambda_phi, He_flux_quiet, 0.0, 95000)[1]

array([0.00000000e+00, 1.61451735e-15, 1.03149719e-12, 3.71697771e-11,
       5.42477828e-10, 2.28185118e-08, 1.41646989e-05, 4.69017289e-05,
       1.12526477e-04, 1.68627367e-04, 3.16445400e-04, 3.58745754e-04,
       2.66215971e-04, 1.50239809e-04, 8.86908196e-05, 2.64240161e-03,
       1.15857765e-02, 6.45692129e-02, 3.36537171e-02, 8.15761798e-02,
       1.89705788e-02, 6.96179880e-02, 3.02768644e-01, 6.80465244e-02,
       2.97071192e-02, 9.61158055e-02, 1.36851872e-02, 2.46569089e-02,
       1.80669873e-02, 2.44538385e-02, 5.37828019e-02, 8.45777469e-02,
       4.34125775e-23, 6.49394755e-23, 9.27450520e-24, 1.82978633e-22,
       9.90237306e-23, 2.15268980e-22, 2.06299439e-23, 6.08134867e-23,
       2.85231398e-23, 9.31038337e-23, 2.29620245e-22, 1.44768389e-22,
       1.88360357e-22, 6.69127745e-22, 5.99165326e-22, 1.68447976e-23,
       4.19774510e-23, 1.04764237e-22, 1.06558145e-22, 1.86566449e-23,
       1.86566449e-23, 9.50771326e-22, 1.35619457e-22, 2.24238520e-23,
      

#### Active Sun

Calculate flux in photons/second and print photodestruction rate

In [11]:
He_flux_active = cs_interp * 1e-18 * phi * ratio_interp

print("Photodestruction rate:", np.sum(He_flux_active))


Photodestruction rate: 1.6803216302077845e-07


Print flux and copy it to _solar_spectrum.jl_ accordingly

In [12]:
He_flux_active

array([0.00000000e+00, 5.73300000e-22, 3.66275000e-19, 1.31986400e-17,
       1.92628800e-16, 8.10264000e-15, 5.02975200e-12, 1.66543650e-11,
       1.56051400e-11, 2.20184750e-11, 4.11586205e-11, 4.61956903e-11,
       3.45108863e-11, 1.96062713e-11, 1.21257858e-11, 4.01078405e-10,
       2.91273840e-09, 1.91110325e-08, 6.42652539e-09, 2.30184974e-08,
       2.88195872e-09, 8.65422135e-09, 4.73335014e-08, 9.77979821e-09,
       3.62188630e-09, 1.53356436e-08, 1.64424010e-09, 3.15349531e-09,
       2.41711200e-09, 3.27158400e-09, 7.14588242e-09, 1.07100542e-08,
       5.41049746e-30, 7.40390413e-30, 1.00717661e-30, 1.91978701e-29,
       1.02407621e-29, 2.29338134e-29, 2.23050971e-30, 6.69335930e-30,
       3.44523552e-30, 1.02397078e-29, 3.15008000e-29, 2.00407286e-29,
       2.54454549e-29, 8.98930000e-29, 8.04940000e-29, 2.26299000e-30,
       5.63940000e-30, 1.14935663e-29, 1.16749334e-29, 2.04139267e-30,
       2.03868926e-30, 1.23916585e-28, 1.77798931e-29, 2.94339392e-30,
      

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify _solar_spectrum.jl_ accordingly

In [13]:
normalize_flux_distribution(lambda_phi, He_flux_active, 0.0, 95000)[1]

array([0.00000000e+00, 3.41184681e-15, 2.17979102e-12, 7.85482955e-11,
       1.14638053e-09, 4.82207683e-08, 2.99332694e-05, 9.91141499e-05,
       9.28699583e-05, 1.31037264e-04, 2.44944895e-04, 2.74921714e-04,
       2.05382622e-04, 1.16681658e-04, 7.21634811e-05, 2.38691449e-03,
       1.73344100e-02, 1.13734372e-01, 3.82458053e-02, 1.36988640e-01,
       1.71512326e-02, 5.15033622e-02, 2.81693103e-01, 5.82019420e-02,
       2.15547204e-02, 9.12661202e-02, 9.78527010e-03, 1.87672125e-02,
       1.43848175e-02, 1.94699868e-02, 4.25268728e-02, 6.37381202e-02,
       3.21991776e-23, 4.40624223e-23, 5.99395133e-24, 1.14251163e-22,
       6.09452495e-23, 1.36484664e-22, 1.32743022e-23, 3.98337983e-23,
       2.05034289e-23, 6.09389752e-23, 1.87468872e-22, 1.19267218e-22,
       1.51432050e-22, 5.34974962e-22, 4.79039242e-22, 1.34676002e-23,
       3.35614319e-23, 6.84009901e-23, 6.94803491e-23, 1.21488210e-23,
       1.21327324e-23, 7.37457539e-22, 1.05812440e-22, 1.75168484e-23,
      

### Method 2: Direct Photon Flux Data per Second and per Wavelength Bin 
*(Multiplication by cross section has already been performed)*

In the past, a cross-sectional database for all the species analyzed in Huebner (2015) was available at [http://phidrates.space.swri.edu/](http://phidrates.space.swri.edu/).  

This website appears to no longer be publicly accessible. Cross-sectional data may still be found in other publications (see Method 1).  

If these sources are not available, Huebner (1992) provides plots of the flux of solar photons per second and per wavelength bin for individual species and reactions. This data already accounts for the product of the species- and wavelength-dependent cross section with the solar flux, and it can be extracted using tools such as [Plot Digitizer](https://plotdigitizer.com/app).  

**Example:** Helium flux in photons/s per wavelength bin from Huebner (1992)

#### Quiet Sun

Flux data should be present in csv file with 2 columns: 'Wavelength (A)' and 'Flux(photon.s-1.binwvl)'

In [15]:
df = pd.read_csv("Helium_quiet.csv")

Interpolate flux in photons/s to match with Huebner 1992 standard wavelength values

In [17]:
He_flux = np.interp(lambda_phi, df['Wavelength (A)'], df['Flux(photon.s-1.binwvl)'])

Calculate photodestruction: sum flux contributions multiplied by wavelength bin

In [18]:
flux_times_bin = pd.Series(He_flux) * ( pd.Series(lambda_phi).shift(-1) - pd.Series(lambda_phi))
print("Photodestruction rate:", flux_times_bin.sum(skipna=True))

Photodestruction rate: 5.5625834779722546e-08


Print flux and copy it to _solar_spectrum.jl_ accordingly

In [19]:
He_flux

array([9.79720011e-18, 9.79720011e-18, 9.79720011e-18, 9.79720011e-18,
       1.18175071e-17, 1.30522145e-17, 5.65074596e-16, 8.21086193e-14,
       2.53359326e-13, 5.08905474e-13, 9.02265903e-13, 1.22192552e-12,
       1.21730626e-12, 9.57161297e-13, 5.14265906e-13, 3.96152478e-13,
       4.55154743e-12, 6.75045534e-11, 8.83655526e-11, 5.94204328e-11,
       9.98806007e-11, 1.55143505e-10, 4.34764166e-10, 1.93657532e-10,
       9.08764856e-11, 5.28297284e-10, 5.28297284e-10, 5.32469184e-11,
       4.03249653e-11, 5.69149583e-11, 1.95521187e-10, 7.03419770e-11,
       9.64651640e-18, 9.64651640e-18, 9.64651640e-18, 9.64651640e-18,
       9.64651640e-18, 9.64651640e-18, 9.64651640e-18, 9.64651640e-18,
       9.64651640e-18, 9.64651640e-18, 9.64651640e-18, 9.64651640e-18,
       9.64651640e-18, 9.64651640e-18, 9.64651640e-18, 9.64651640e-18,
       9.64651640e-18, 9.64651640e-18, 9.64651640e-18, 9.64651640e-18,
       9.64651640e-18, 9.64651640e-18, 9.64651640e-18, 9.64651640e-18,
      

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify _solar_spectrum.jl_ accordingly

In [20]:
normalize_flux_distribution(lambda_phi, He_flux, 0.0, 95000)[1]

array([3.66501769e-09, 3.66501769e-09, 3.66501769e-09, 3.66501769e-09,
       4.42079084e-09, 4.88268041e-09, 2.11387781e-07, 3.07158718e-05,
       9.47787531e-05, 1.90375571e-04, 3.37527095e-04, 4.57108008e-04,
       4.55379998e-04, 3.58062818e-04, 1.92380846e-04, 1.48195997e-04,
       1.70268053e-03, 2.52526619e-02, 3.30565171e-02, 2.22284872e-02,
       3.73641616e-02, 5.80373663e-02, 1.62640177e-01, 7.24450119e-02,
       3.39958277e-02, 1.97629820e-01, 1.97629820e-01, 1.99190479e-02,
       1.50850967e-02, 2.12912186e-02, 7.31421836e-02, 2.63141088e-02,
       3.60864868e-09, 3.60864868e-09, 3.60864868e-09, 3.60864868e-09,
       3.60864868e-09, 3.60864868e-09, 3.60864868e-09, 3.60864868e-09,
       3.60864868e-09, 3.60864868e-09, 3.60864868e-09, 3.60864868e-09,
       3.60864868e-09, 3.60864868e-09, 3.60864868e-09, 3.60864868e-09,
       3.60864868e-09, 3.60864868e-09, 3.60864868e-09, 3.60864868e-09,
       3.60864868e-09, 3.60864868e-09, 3.60864868e-09, 3.60864868e-09,
      

#### Repeat for Active Sun